In [6]:
import pickle

import pandas as pd
import os
import pickle
import os


In [12]:
df = pd.read_csv("../casia.csv")
df.head()

,usage_type,ground_truth,subject_number,video_name,video_frames,frames_present,attack_category,directory_path
0,test,spoof,1,HR_4.avi,115.0,115,HR_R,test/spoof/1/HR_4
1,test,spoof,1,HR_2.avi,249.0,247,HR_W,test/spoof/1/HR_2
2,test,spoof,1,HR_3.avi,93.0,91,HR_C,test/spoof/1/HR_3
3,test,spoof,1,3.avi,213.0,213,W1,test/spoof/1/3
4,test,spoof,1,4.avi,198.0,198,W2,test/spoof/1/4


In [13]:
directory_paths = df['directory_path'].tolist()

In [14]:
def fix_directory(row):
    prev_path = os.path.dirname(row['directory_path'])
    folder_name = os.path.basename(row['directory_path'])
    return f"{prev_path}/{row['subject_number']}-{folder_name}"

In [15]:
df['directory_path'] = df.apply(lambda row: fix_directory(row), axis=1)
df.head()

,usage_type,ground_truth,subject_number,video_name,video_frames,frames_present,attack_category,directory_path
0,test,spoof,1,HR_4.avi,115.0,115,HR_R,test/spoof/1/1-HR_4
1,test,spoof,1,HR_2.avi,249.0,247,HR_W,test/spoof/1/1-HR_2
2,test,spoof,1,HR_3.avi,93.0,91,HR_C,test/spoof/1/1-HR_3
3,test,spoof,1,3.avi,213.0,213,W1,test/spoof/1/1-3
4,test,spoof,1,4.avi,198.0,198,W2,test/spoof/1/1-4


In [2]:
# /home/jarred/Documents/Datasets/CASIA/test/real/1/1/frame_355.png
# /home/jarred/Documents/Datasets/SIW/test/real/75/075-1-1-1-1/frame_355.png
csv_path = "../test_LOO_SGS8_results.csv"
df = pd.read_csv(csv_path)
df.head(10)

,file_paths,ground_truth,predicted
0,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
1,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
2,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
3,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
4,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
5,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
6,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
7,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
8,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0
9,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0


In [4]:
def categorise_video(row):
    return os.path.basename(os.path.dirname(row['file_paths']))


In [5]:
df['video_name'] = df.apply(lambda row: categorise_video(row), axis=1)
df.head(10)

,file_paths,ground_truth,predicted,video_name
0,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
1,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
2,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
3,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
4,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
5,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
6,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
7,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
8,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1
9,/home/jarred/Documents/Datasets/SIW/test/real/...,0,0,/SIW/test/real/75/075-1-1-1-1


In [8]:
video_names = df['video_name'].unique()

In [14]:
temp_df = df.query(f"video_name == '{video_names[0]}'")
spoof_pred_count = temp_df[(temp_df.predicted == 1)].count()["predicted"]
real_pred_count = temp_df[(temp_df.predicted == 0)].count()["predicted"]
ground_truth = temp_df['ground_truth'].tolist()[0]
if spoof_pred_count > real_pred_count:
    predicted = 1
else:
    predicted = 0

In [3]:
from Antispoofing.AntispoofHelpers.spoof_metric import determine_spoof_metrics


def video_based_results(single_frame):
    temp_single = single_frame.copy()
    def categorise_video(row):
        return os.path.basename(os.path.dirname(row['file_paths']))
    temp_single['video_name'] = temp_single.apply(lambda row: categorise_video(row), axis=1)
    video_names = temp_single['video_name'].unique()
    video_list = []
    for name in video_names:
        temp_df = temp_single.query(f"video_name == '{name}'")
        real = 0
        spoof = 1
        spoof_pred_count = temp_df[(temp_df.predicted == 1)].count()["predicted"]
        real_pred_count = temp_df[(temp_df.predicted == 0)].count()["predicted"]
        if spoof_pred_count > real_pred_count:
            predicted = 1
        else:
            predicted = 0
        ground_truth = temp_df['ground_truth'].tolist()[0]
        video_list.append({"video_name": name, f'spoof({spoof})_pred_count': spoof_pred_count, f'real({real}_pred_count': real_pred_count, "predicted": predicted, "ground_truth": ground_truth})
    multi_frame = pd.DataFrame.from_dict(video_list)
    predicted = multi_frame['predicted'].tolist()
    ground_truth = multi_frame['ground_truth'].tolist()
    metric_dic = determine_spoof_metrics(ground_truth, predicted, "ASUS", 1,1, must_show=False)
    metric_dic = dict(("{}_{}".format("Multi",k),v) for k,v in metric_dic.items())
    return metric_dic
multi_metric_dic = video_based_results(df)


In [18]:
multi_frame_df = pd.DataFrame.from_dict(video_list)


In [20]:
print(multi_frame_df['predicted'].tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [19]:
d = {'a': 1, 'b': 2}
multi_d = dict(("{}_{}".format("Multi",k),v) for k,v in d.items())

In [4]:
print(os.path.basename(os.path.dirname("/home/jarred/Documents/Datasets/SIW/test/real/75/075-1-1-1-1/frame_355.png")))

075-1-1-1-1


In [2]:
folder = "/home/jarred/Documents/GANModels/SIW_KF_90/ASUS/00005-stylegan3-r-ASUS-gpus2-batch32-gamma32"
name = "metric-fid50k_full.jsonl"


In [5]:
import json

with open(os.path.join(folder,name), 'r') as f:
    lines = f.readlines()
    results = []
    for line in lines:
        result = json.loads(line)
        kimg = int(os.path.splitext(result['snapshot_pkl'])[0].split("-")[-1])
        results.append({'fid': result["results"]["fid50k_full"], "model":result["snapshot_pkl"], 'fake_image': f"fakes{kimg:06d}.png"})

In [6]:
df = pd.DataFrame.from_dict(results)
df.head()

,fid,model,fake_image
0,57.085854,network-snapshot-000000.pkl,fakes000000.png
1,59.591817,network-snapshot-000040.pkl,fakes000040.png
2,62.606670,network-snapshot-000080.pkl,fakes000080.png
3,62.217534,network-snapshot-000120.pkl,fakes000120.png
4,97.319817,network-snapshot-000160.pkl,fakes000160.png


In [ ]:
metrics = []
for r in results:
    if best_fid > results[i]["results"]["fid50k_full"]:
        best_fid = results[i]["results"]["fid50k_full"]
        best_pickle_path = results[i]["snapshot_pkl"]
        metrics.

In [2]:
with open("/home/jarred/Documents/Datasets/SIW_KF/test/spoof/90/090-2-3-4-2/k_vs_sil.pkl", "rb") as f:
    dic = pickle.load(f)


In [3]:
print(dic)

[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,

In [4]:
with open("/home/jarred/Documents/Datasets/SIW_KF_OLD/test/spoof/90/090-2-3-4-2/index_sil.pkl", "rb") as f:
    dic = pickle.load(f)

In [5]:
print(dic)

[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,